Get data from github

In [40]:
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_dataset.py

--2023-06-26 11:25:15--  https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 891 [text/plain]
Saving to: ‘bert_dataset.py.1’

bert_dataset.py.1   100%[===================>]     891  --.-KB/s    in 0s      

2023-06-26 11:25:15 (65.6 MB/s) - ‘bert_dataset.py.1’ saved [891/891]



Installing Packages

In [41]:
!pip install sentencepiece==0.1.95

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
import pandas as pd

In [44]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
RANDOM_STATE = 12

In [48]:
BACKTEST_CCY = 'USDRUB'
BACKTEST_DAYS = 5
YEARS = [2021, 2022]

PATH_TEXTS = 'drive/MyDrive/data/telegram'
PATH_OPT_PNL = 'drive/MyDrive/data/pnl/option'

N_EPOCHS = 10

In [49]:
import os

# Get all available data_sources
sources = os.listdir(PATH_TEXTS)
sources

['bitkogan.csv',
 'War_Wealth_Wisdom.csv',
 'themovchans.csv',
 'vts.csv',
 'sky_bond.csv',
 'Alfa_Wealth.csv',
 'rshb_invest.csv',
 'headlines_QUANTS.csv',
 'signal.csv',
 'mmi.csv',
 'cbonds.csv']

In [50]:
# Create target variable dataframe
pnl = pd.DataFrame()
for year in YEARS:
    pnl_year = pd.read_csv(f'{PATH_OPT_PNL}/Backtest_{BACKTEST_CCY}_{BACKTEST_DAYS}_days_{year}.txt')
    print(f'{PATH_OPT_PNL}/Backtest_{BACKTEST_CCY}_{BACKTEST_DAYS}_days_{year}.txt')
    pnl = pnl.append(pnl_year)
pnl['date_start'] = pd.to_datetime(pnl['date_start']).dt.strftime('%Y-%m-%d')
pnl.dropna(subset=['pnl'], inplace=True)

drive/MyDrive/data/pnl/option/Backtest_USDRUB_5_days_2021.txt
drive/MyDrive/data/pnl/option/Backtest_USDRUB_5_days_2022.txt


In [51]:
import datetime as dt
from typing import List, Tuple, Union


def _binary_search_by_date(array: List[Tuple[dt.datetime, float]], date_x: dt.datetime) -> Union[int, None]:
    """
    Searches for the index of date_x in the array via binary search.

        Parameters:
            array (list) : A sorted array of (date, float_value) tuples
            date_x (datetime.datetime) : Date to search for

        Returns:
            index_x (int): Index of the searched date in the array.
    """
    left = 0
    right = len(array) - 1

    while left <= right:
        mid = left + (right - left) // 2

        if array[mid - 1][0] <= date_x <= array[mid][0]:
            return mid
        elif date_x > array[mid - 1][0] and date_x > array[mid][0]:
            left = mid + 1
        else:
            right = mid - 1

    return None


def binary_search_time_series(time_series: List[Tuple[dt.datetime, float]], date_start: dt.datetime,
                              date_end: dt.datetime) -> Union[List[Tuple[dt.datetime, float]], None]:
    """
    Searches for the part of the time series that is contained inside [date_start; date_end] period via binary search.

        Parameters:
            time_series (list) : A sorted array of (date, float_value) tuples
            date_start (datetime.datetime) : Starting date of the searched period
            date_end (datetime.datetime) : Ending date of the searched period

        Returns:
            time_series_data (list): Part of the time series that is contained inside [date_start; date_end] period.
    """

    if date_start <= date_end:
        left_index = _binary_search_by_date(time_series, date_start)

        if left_index is None:
            return None

        right_index = _binary_search_by_date(time_series[left_index:], date_end)

        if right_index is None:
            return None

        right_index += left_index
    else:
        left_index = _binary_search_by_date(time_series, date_end)

        if left_index is None:
            return None

        right_index = _binary_search_by_date(time_series[left_index:], date_start)

        if right_index is None:
            return None

        right_index += left_index

    return time_series[left_index:right_index]

In [52]:
pnl_sign_ts = [(pd.to_datetime(row['date_start']).to_pydatetime(), row['pnl']) for  _, row in pnl.iterrows()]
pnl_sign_ts

[(datetime.datetime(2021, 1, 4, 0, 0), 291264.0979135564),
 (datetime.datetime(2021, 1, 5, 0, 0), -411993.8303199067),
 (datetime.datetime(2021, 1, 6, 0, 0), 521491.6867951669),
 (datetime.datetime(2021, 1, 8, 0, 0), 62842.63411628307),
 (datetime.datetime(2021, 1, 11, 0, 0), -537598.7062168794),
 (datetime.datetime(2021, 1, 12, 0, 0), -359059.05784320267),
 (datetime.datetime(2021, 1, 13, 0, 0), 181132.6897838658),
 (datetime.datetime(2021, 1, 14, 0, 0), 47895.30725670268),
 (datetime.datetime(2021, 1, 15, 0, 0), 30290.59373304957),
 (datetime.datetime(2021, 1, 18, 0, 0), 1157975.3869589714),
 (datetime.datetime(2021, 1, 19, 0, 0), 1258409.487166846),
 (datetime.datetime(2021, 1, 20, 0, 0), 1849608.792259631),
 (datetime.datetime(2021, 1, 21, 0, 0), 1996906.8565119945),
 (datetime.datetime(2021, 1, 22, 0, 0), 762996.8728368632),
 (datetime.datetime(2021, 1, 25, 0, 0), 560685.9672308444),
 (datetime.datetime(2021, 1, 26, 0, 0), -95365.18586283142),
 (datetime.datetime(2021, 1, 27, 0, 0

In [53]:
initial_date = pnl_sign_ts[0][0]

def get_pnl_sign(row):
    date_x = pd.to_datetime(row['date']).to_pydatetime()
    pnl_sign_key = _binary_search_by_date(pnl_sign_ts, date_x)

    if pnl_sign_key is not None and date_x >= initial_date:
        return pnl_sign_ts[_binary_search_by_date(pnl_sign_ts, date_x)][1]

In [54]:
# Generate dataframes
df = pd.DataFrame()

for s in sources:
    if s != '.gitignore':
        source_data = pd.read_csv(f'{PATH_TEXTS}/{s}')
        df = df.append(source_data)

df.head()

,id,date,text
0,3,2018-01-15T16:12:18,из российских биржевых идеек я бы посмотрел се...
1,5,2018-01-16T00:47:25,рекомендовал пару недель назад xmg. cn\nпервые...
2,7,2018-01-16T14:48:39,несмотря на страшно. ужас ужас. выкупаю паден...
3,8,2018-01-16T15:04:23,"январское ралли....в принципе, это не нечто не..."
4,10,2018-01-16T21:36:35,все идет по плану


In [55]:
df['pnl_sign'] = df.apply(lambda row: get_pnl_sign(row), axis=1)
df.dropna(subset=['pnl_sign'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,id,date,text,pnl_sign
0,10186,2021-01-05T09:37:44,Почему просели котировки нефти. Все очень прос...,521491.686795
1,10187,2021-01-05T09:41:54,Brent,521491.686795
2,10188,2021-01-05T09:42:23,Light,521491.686795
3,10189,2021-01-05T10:01:04,По итогам первого торгового дня года получил з...,521491.686795
4,10190,2021-01-05T10:09:56,Что-то непонятное происходит с рублем. USD/RUB...,521491.686795


In [56]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [57]:
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

from functools import lru_cache
from tqdm.notebook import tqdm
from tqdm.gui import tqdm as tqdm_gui

m = MorphAnalyzer()
regex = re.compile("[а-яa-zёЁ]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]

mystopwords = stopwords.words('russian')
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)

    return ' '.join(remove_stopwords(lemmas))

In [58]:
lemmas = list(tqdm(map(clean_text, df['text']), total=len(df['text'].to_list())))

  0%|          | 0/45412 [00:00<?, ?it/s]

In [59]:
df['lemmas'] = lemmas
df = df[df['lemmas'] != '']
df.head()

,id,date,text,pnl_sign,lemmas
0,10186,2021-01-05T09:37:44,Почему просели котировки нефти. Все очень прос...,521491.686795,почему просесть котировка нефть очень просто п...
1,10187,2021-01-05T09:41:54,Brent,521491.686795,brent
2,10188,2021-01-05T09:42:23,Light,521491.686795,light
3,10189,2021-01-05T10:01:04,По итогам первого торгового дня года получил з...,521491.686795,итог первый торговый день получить звонок встр...
4,10190,2021-01-05T10:09:56,Что-то непонятное происходит с рублем. USD/RUB...,521491.686795,непонятный происходить рубль взлететь минута п...


In [60]:
df.shape

(45362, 5)

In [61]:
!pip install fast_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
from fast_ml.model_development import train_valid_test_split

X_train, X_train['pnl_sign'], X_valid, X_valid['pnl_sign'], X_test, X_test['pnl_sign'] = train_valid_test_split(df[['lemmas', 'pnl_sign']], target = 'pnl_sign',
                                                    train_size=0.8, valid_size=0.1, test_size=0.1)

In [65]:
import numpy as np

pnl_mean = X_train['pnl_sign'].mean()
pnl_std = np.std(X_train['pnl_sign'])

X_train['pnl_sign'] = (X_train['pnl_sign'] - pnl_mean) / pnl_std
X_valid['pnl_sign'] = (X_valid['pnl_sign'] - pnl_mean) / pnl_std
X_test['pnl_sign'] = (X_test['pnl_sign'] - pnl_mean) / pnl_std

In [66]:
def classify(data: pd.DataFrame, column_name: str = 'pnl', thr: float = 0.0, include_small: bool = False) -> pd.DataFrame:
    data[column_name] = data[column_name].apply(lambda x: 1 if x >= thr else (0 if x <= -thr else 2))
    if not include_small:
        data = data[data[column_name] != 2]
    return data

In [67]:
X_train = classify(X_train, 'pnl_sign')
X_valid = classify(X_valid, 'pnl_sign')
X_test = classify(X_test, 'pnl_sign')

In [68]:
X_train.shape[0], X_valid.shape[0], X_valid.shape[0]

(36289, 4536, 4536)

In [69]:
X_train['pnl_sign'].sum() / X_train['pnl_sign'].shape[0], X_valid['pnl_sign'].sum() / X_valid.shape[0], X_test['pnl_sign'].sum() / X_test.shape[0]

(0.34239025600044093, 0.3544973544973545, 0.34075380207185363)

In [70]:
N_TOKENS_PER_BATCH = 12000

batch_size = int(round(N_TOKENS_PER_BATCH / df['lemmas'].apply(lambda x: len(x.split(' '))).mean(), 0))
batch_size

228

Initialize BERT classifier

In [71]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=len(np.unique(X_train['pnl_sign'])),
        epochs=N_EPOCHS,
        model_save_path='/content/bert.pt'
)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny a

Prepare data and helpers for train and evlauation

In [72]:
classifier.preparation(
        X_train=X_train['lemmas'].tolist(),
        y_train=X_train['pnl_sign'].tolist(),
        X_valid=X_valid['lemmas'].tolist(),
        y_valid=X_valid['pnl_sign'].tolist(),
        batch_size=20
    )

Train loop

In [ ]:
classifier.train()

Epoch 1/10
Train loss 0.6867375075652908 accuracy 0.5648819201410896
Val loss 0.6830543825804924 accuracy 0.6027336860670194
----------
Epoch 2/10
Train loss 0.6683365448447298 accuracy 0.6122516465044504
Val loss 0.6776107336981181 accuracy 0.5742945326278659
----------
Epoch 3/10
Train loss 0.654541658681943 accuracy 0.6285100168095015
Val loss 0.6797975761249727 accuracy 0.5983245149911817
----------
Epoch 4/10
Train loss 0.6382847902531794 accuracy 0.6493703326076773
Val loss 0.6880929747079437 accuracy 0.5795855379188712
----------
Epoch 5/10
Train loss 0.6198124358969287 accuracy 0.6686048113753479
Val loss 0.706333104603091 accuracy 0.5923721340388006
----------
Epoch 6/10
Train loss 0.6050517077616752 accuracy 0.6800407837085618
Val loss 0.7112883938566703 accuracy 0.5943562610229276
----------
Epoch 7/10
Train loss 0.5904886507462536 accuracy 0.693984402987131
Val loss 0.7304417957030729 accuracy 0.5945767195767195
----------
Epoch 8/10
Train loss 0.5794091269497044 accuracy 0

Check test data

In [ ]:
predictions = [classifier.predict(t) for t in X_test['lemmas'].tolist()]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(X_test['pnl_sign'].tolist(), predictions, average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(X_test['pnl_sign'].tolist(), predictions)
cm

In [ ]:
accuracy_score(X_test['pnl_sign'].tolist(), predictions)

In [ ]:
max(X_test['pnl_sign'].sum() / X_test.shape[0], 1 - X_test['pnl_sign'].sum() / X_test.shape[0])

In [ ]:
df['lemmas'].apply(lambda x: len(x.split(' '))).mean()

In [ ]:
12000 / df['lemmas'].apply(lambda x: len(x.split(' '))).mean()